<a href="https://colab.research.google.com/github/shayanAbbasi1995/BGSE-Macro-Finance/blob/main/Edgar_scrap_first_10K_links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data

## Libraries

In [ ]:
# import our libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

import time

##Connect to Google Drive


In [ ]:
# Run this if running in Google Collab
# Mount google drive if running from Google Collab
from google.colab import drive
drive.mount('/content/drive')

# Set current directory if running from Google Collab

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/edgar/SEC_Filing')# here use your path to current notebook

Mounted at /content/drive


## Read tickers and CIK

In [ ]:
symbs = pd.read_stata("../Firm-code-info.dta")
cik_firstlink_df = pd.read_csv('first_link_df.csv')

In [ ]:
cik_firstlink_df

,Unnamed: 0,cik,first_links
0,0,1890567,[]


In [ ]:
# How many tickers do we have?
print(f' we have {len(symbs.tic.unique())} tickers, {len(symbs.cik.unique())} CIKs, {len(symbs.gvkey.unique())} GVKEYs, {len(symbs.conm.unique())} company names')

 we have 41671 tickers, 30033 CIKs, 41692 GVKEYs, 41689 company names


In [ ]:
firm_cik = list(symbs.cik.unique())

So, it's best to use tickers or GVKEYs

# Scraping functions and Error handling

In [ ]:
# You can find company's CIK number at https://www.sec.gov/edgar/searchedgar/companysearch.html
company_CIKs = list(filter(None, list(symbs.cik.unique())))
# Enter what forms(s) you want to extract using the '10-K', '10-Q', '8-K' format.


In [ ]:
!pip install requests-random-user-agent

     |████████████████████████████████| 45 kB 1.6 MB/s 
  Created wheel for requests-random-user-agent: filename=requests_random_user_agent-2022.5.3-py3-none-any.whl size=47121 sha256=d4f3dca2a07d8b512c421418f299c1a00fdc2535c617b882a9cc48d652d64f63
  Stored in directory: /root/.cache/pip/wheels/b2/4c/bd/401688226482772eedcc3bd668dc012221fff13d96e0a549c8
Successfully built requests-random-user-agent


In [ ]:
import requests_random_user_agent
import re

# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"
s = requests.Session()
print(s.headers['User-Agent'])

headers = {"User-Agent": 'abbasi.shayan1995@gmail.com'}


ciks_list = []
first_links_list = []
for cik in firm_cik:
  # define our parameters dictionary
  param_dict = {'action':'getcompany',
                'CIK':cik,
                'type':'10-k',
                'dateb':'20220101',
                'owner':'exclude',
                'start':'',
                'output':'',
                'count':'100'}

  # request the url, and then parse the response.
  response = requests.get(url = endpoint, params = param_dict, headers=headers)
  #response.raise_for_status()
  soup = BeautifulSoup(response.content, 'html.parser')
  print('Request Successful')
  print(response.url)
  # find the document table with our data
  doc_table = soup.find_all('table', class_='tableFile2')

  # define a base url that will be used for link building.
  base_url_sec = r"https://www.sec.gov"


  temp_links = []
  if len(doc_table) != 0:
    hrefs = doc_table[0].find_all("a", id= "documentsbutton", href=True)
    for tag in hrefs:
      temp_links.append(base_url_sec + tag['href'])
  ciks_list.append(cik)
  first_links_list.append(temp_links)

d = {'cik' : ciks_list, 'first_links' : first_links_list}
first_links_df = pd.DataFrame(d, columns = ['cik', 'first_links'])
first_links_df.to_csv('first_link_df.csv')


Streaming output truncated to the last 5000 lines.
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001379785&type=10-k&dateb=20220101&owner=exclude&start=&output=&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001369774&type=10-k&dateb=20220101&owner=exclude&start=&output=&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001102392&type=10-k&dateb=20220101&owner=exclude&start=&output=&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001266677&type=10-k&dateb=20220101&owner=exclude&start=&output=&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001373641&type=10-k&dateb=20220101&owner=exclude&start=&output=&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001393435&type=10-k&dateb=20220101&owner=exclude&start=&output=&count=100
Request